# Section 1 Project

# 1. 데이터 탐색

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv',index_col = 0)

In [ ]:
df.head()

Data Description  
Name : 게임의 이름입니다.  
Platform : 게임이 지원되는 플랫폼의 이름입니다.  
Year : 게임이 출시된 연도입니다.  
Genre : 게임의 장르입니다.  
Publisher : 게임을 제작한 회사입니다.  
NA_Sales : 북미지역에서의 출고량입니다.  
EU_Sales : 유럽지역에서의 출고량입니다.  
JP_Sales : 일본지역에서의 출고량입니다.  
Other_Sales : 기타지역에서의 출고량입니다.

In [ ]:
df.info()

In [ ]:
df[df.duplicated()] # 중복확인

In [ ]:
df[df['Name'] == 'Wii de Asobu: Metroid Prime']

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df['Year'].astype(object).unique()

In [ ]:
df['NA_Sales'].astype(object).unique()

# 2.데이터 전처리

#### 1.Sales열 단위 맞추기

In [ ]:
# Sales 컬럼들에서 K를 *1e-3으로 M을 *1로 replace하여 eval함수를 이용하여 value들을 연산한다.
df['NA_Sales'] = df['NA_Sales'].replace({'K': '*1e-3', 'M': '*1'}, regex=True).map(pd.eval).round(2) #round로 floating point 방지
df['EU_Sales'] = df['EU_Sales'].replace({'K': '*1e-3', 'M': '*1'}, regex=True).map(pd.eval).round(2)
df['JP_Sales'] = df['JP_Sales'].replace({'K': '*1e-3', 'M': '*1'}, regex=True).map(pd.eval).round(2)
df['Other_Sales'] = df['Other_Sales'].replace({'K': '*1e-3', 'M': '*1'}, regex=True).map(pd.eval).round(2)

#### 2.Year열 단위 맞추기

In [ ]:
# Year열 value 단위 맞추기
# Year열의 NaN값과 관계없이 사용될 수 있는 데이터가 있을 수 있기 때문에 dropna로 처리하지 않는다.
def float_to_year(x):
    if len(str(x)) < 4:         # Year의 Value를 string으로 봤을때 길이가 4 미만인 것만 선택
        if x < 22:              # 22년도 이전은 +2000년
            return x + 2000
        elif x > 23:            # 23년도 이후는 +1900년
            return x + 1900
    else:
        return x                # 해당사항 없을 경우 그대로 리턴

df['Year'] = df['Year'].astype(int)
df['Year'] = df['Year'].apply(float_to_year)
df

In [ ]:
df['Year'].unique()

#### 3.Total Sales 추가

In [ ]:
sales = df.loc[:,'NA_Sales':'Other_Sales']
df['Total_Sales'] = sales.sum(axis=1)
df

#### 4.데이터 부족한 연도 제거
2015년도 이후 데이터는 데이터 분석에 적절하지 않을 것 같다고 판단

In [ ]:
total_sales_group = df[['Year','Total_Sales']].groupby(['Year'])
total_sales_group = total_sales_group.sum().reset_index()

In [ ]:
plt.figure(figsize = (20,8))
sns.barplot(
    data= total_sales_group,
    x= "Year",
    y= "Total_Sales"
)

In [ ]:
df = df.drop(df[df['Year'] > 2015].index).reset_index(drop = True)

In [ ]:
df['Year'].unique()

In [ ]:
df.shape

# 3.데이터 분석

## 1.지역에 따라서 선호하는 게임 장르가 다를까?

In [ ]:
df_genre = df[['Genre','NA_Sales','EU_Sales','JP_Sales','Other_Sales']]
df_genre = df_genre.dropna().reset_index(drop=True)
df_genre.head()

In [ ]:
df_genre_group = df_genre.groupby(['Genre']).sum().reset_index()
df_genre_group

In [ ]:
genre_melt = df_genre_group.melt(id_vars='Genre', value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales'], var_name='Country', value_name='Sales')
genre_melt

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(x="Country", y="Sales",hue="Genre", data=genre_melt)

#### 등분산성
귀무가설 : 등분산이다.  
대립가설 : 이분산이다.

In [ ]:
NA_Sales = df_genre_group['NA_Sales']
EU_Sales = df_genre_group['EU_Sales']
JP_Sales = df_genre_group['JP_Sales']
Other_Sales = df_genre_group['Other_Sales']

In [ ]:
plt.xlabel("Sales")
sns.kdeplot(data=NA_Sales, color="red", shade=True)
sns.kdeplot(data=EU_Sales, color="blue", shade=True)
sns.kdeplot(data=JP_Sales, color="green", shade=True)
sns.kdeplot(data=Other_Sales, color="purple", shade=True)
plt.show()

In [ ]:
stats.levene(NA_Sales, EU_Sales, JP_Sales, Other_Sales) # pvalue < 0.05, 이분산이다.

In [ ]:
stats.levene(NA_Sales, EU_Sales) # pvalue > 0.05, 등분산이다.

In [ ]:
stats.levene(NA_Sales, JP_Sales) # pvalue < 0.05, 이분산이다.

In [ ]:
stats.levene(NA_Sales, Other_Sales) # pvalue < 0.05, 이분산이다.

In [ ]:
stats.levene(EU_Sales, JP_Sales) # pvalue > 0.05, 등분산이다.

In [ ]:
stats.levene(EU_Sales, Other_Sales) # pvalue < 0.05, 이분산이다.

In [ ]:
stats.levene(JP_Sales, Other_Sales) # pvalue > 0.05, 등분산이다.

In [ ]:
stats.levene(EU_Sales, JP_Sales, Other_Sales) # pvalue > 0.05, 등분산이다.

#### Anova 검정
귀무가설: 집단의 평균이 유의미하다.  
대립가설: 집단의 평균(최소 1개 집단 이상)은 유의미하지 않다.

In [ ]:
stats.f_oneway(NA_Sales, EU_Sales, JP_Sales, Other_Sales) # pvalue < 0.05 귀무가설 기각

In [ ]:
stats.f_oneway(NA_Sales, JP_Sales) # pvalue < 0.05 귀무가설 기각, 두 나라의 선호하는 게임 장르가 다르다.

In [ ]:
stats.f_oneway(NA_Sales, EU_Sales) # pvalue > 0.05 귀무가설 채택, 두 나라의 선호하는 게임 장르가 유사하다.

In [ ]:
stats.f_oneway(NA_Sales, Other_Sales) # pvalue < 0.05 귀무가설 기각, 두 나라의 선호하는 게임 장르가 다르다.

In [ ]:
stats.f_oneway(EU_Sales, JP_Sales) # pvalue > 0.05  귀무가설 채택, 두 나라의 선호하는 게임 장르가 유사하다.

In [ ]:
stats.f_oneway(EU_Sales, Other_Sales) # pvalue < 0.05 귀무가설 기각, 두 나라의 선호하는 게임 장르가 다르다.

In [ ]:
stats.f_oneway(JP_Sales, Other_Sales) # pvalue > 0.05 귀무가설 채택, 두 나라의 선호하는 게임 장르가 유사하다.

In [ ]:
stats.f_oneway(EU_Sales, JP_Sales, Other_Sales) # pvalue < 0.05 귀무가설 기각, 세 나라의 선호하는 게임 장르가 다르다.

## 2.연도별 게임의 트렌드가 있을까?

In [ ]:
def groupby_df(column_name):
    title = df[['Year','Total_Sales',column_name]]
    group = title.groupby(['Year',column_name])
    multi_index = group.sum().reset_index()
    return multi_index

#### 1.연도별 플랫폼 트렌드

In [ ]:
data1 = groupby_df('Platform')

In [ ]:
for i in data1['Platform'].unique():
    data = data1[data1['Platform'] == i]
    plt.figure(figsize = (3,3))
    sns.lineplot(
        data= data,
        x= "Year",
        y= "Total_Sales",
        hue ="Platform"
    )

In [ ]:
plt.figure(figsize = (30,10))
sns.lineplot(
    data= data1,
    x= "Year",
    y= "Total_Sales",
    hue ="Platform"
)

#### 2.연도별 장르 트렌드

In [ ]:
data2 = groupby_df('Genre')

In [ ]:
for i in data2['Genre'].unique():
    data = data2[data2['Genre'] == i]
    plt.figure(figsize = (3,3))
    sns.lineplot(
        data= data,
        x= "Year",
        y= "Total_Sales",
        hue ="Genre"
    )

In [ ]:
plt.figure(figsize = (30,10))
sns.lineplot(
    data= data2,
    x= "Year",
    y= "Total_Sales",
    hue ="Genre"
)

## 3.출고량이 높은 게임에 대한 분석 및 시각화 프로세스

In [ ]:
df['Total_Sales'].sort_values(ascending=False).describe()

### 3-1.2011~2015년 출고량 100만부 이상

In [ ]:
df1 = df[df['Year'] >= 2011]
df1['Total_Sales'].describe()

In [ ]:
df1 = df1[df1['Total_Sales'] >= 1].sort_values('Total_Sales',ascending=False).reset_index(drop=True)
df1

In [ ]:
df1[['Year']].value_counts()

#### 3-1-1.연도별 장르

In [ ]:
plt.figure(figsize=(30, 10))

ax = sns.countplot(x="Year", data=df1, hue='Genre', order=[2011, 2012, 2013, 2014, 2015])
for p in ax.patches:
   ax.annotate(p.get_height(), (p.get_x(), p.get_height()))

#### 3-1-2.연도별 플랫폼

In [ ]:
plt.figure(figsize=(30, 10))
ax = sns.countplot(x="Year", data=df1, hue='Platform', order=[2011, 2012, 2013, 2014, 2015])
for p in ax.patches:
   ax.annotate(p.get_height(), (p.get_x(), p.get_height())) #p.get_height() : 높이 = count값, (p.get_x(), p.get_height()) : display 위치 

# 추가분석

In [ ]:
df2 = df[df['Year'] >= 2010]
df2

In [ ]:
def groupby_df2(column_name):
    title = df2[['Year','Total_Sales',column_name]]
    group = title.groupby(['Year',column_name])
    multi_index = group.sum().reset_index()
    return multi_index

In [ ]:
data1 = groupby_df2('Genre')
data1 = data1.pivot_table(index = 'Genre', columns = 'Year', values = 'Total_Sales', aggfunc='first').reset_index()

In [ ]:
data1

In [ ]:
data2 = data1[[2010,2011,2012,2013,2014,2015]]

In [ ]:
def yearonyear(df):
    new_df = pd.DataFrame()
    for i in range(df.shape[1]-1):
        value = ((df[df.columns[i+1]] / df[df.columns[i]]) * 100).round(1)
        new_df[df.columns[i+1]] = value
    return new_df

In [ ]:
def yearonyear(df):
    new_df = pd.DataFrame()
    for i in range(df.shape[1]-1):
        value = ((df[df.columns[i+1]] / df[df.columns[i]]) * 100).round(1)
        new_df[df.columns[i+1]] = value

    latest_year = df[df.columns[df.shape[1]-1]]
    first_year = df[df.columns[0]]
    l2f = ((latest_year / first_year) * 100).round(1) #마지막 열 데이터 / 첫번째 열 데이터
    new_df['l/f'] = l2f
    return new_df

In [ ]:
yoy = yearonyear(data2)

In [ ]:
yoy.insert(0,'Genre',data1['Genre'])
yoy

In [ ]:
yoy_ = yoy[['Genre',2011,2012,2013,2014,2015]]

In [ ]:
yoy_melt = yoy_.melt(id_vars='Genre', value_vars=[2011,2012,2013,2014,2015], var_name='Year', value_name='yoy')
yoy_melt

In [ ]:
plt.figure(figsize = (30,10))
sns.lineplot(
    data= yoy_melt,
    x= "Year",
    y= "yoy",
    hue ="Genre"
)

# 1. 2011년 이후 100만부 이상 판매

In [ ]:
df3 = df[df['Year'] >= 2011].reset_index(drop=True)
df3

## 1-1. 연도별 국가단위 소비 추이

In [ ]:
year_sales = df3[['Year','NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales','Total_Sales']]

In [ ]:
country_group = year_sales.groupby(['Year']).sum().reset_index()

In [ ]:
country_melt = country_group.melt(id_vars='Year', value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales'], var_name='Country', value_name='Sales')
country_melt.head()

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(
    data = country_melt,
    x = "Country",
    y = "Sales",
    hue = "Year"
)
for p in ax.patches:
   ax.annotate(p.get_height().round(1), (p.get_x(), p.get_height()+1)) #p.get_height() : 높이 = count값, (p.get_x(), p.get_height()) : display 위치 

## 1-2. 추가분석
출고량이 상대적으로 높은 게임들을 제거하고 비교해도 전체적인 출고량이 줄어들고 있는지 확인.

In [ ]:
df[df['Total_Sales'] >= 5].shape

In [ ]:
df_drop = df[df['Total_Sales'] <= 2].reset_index(drop=True)

In [ ]:
df_drop = df_drop[df_drop['Year'] >= 2011]
year_sales = df_drop[['Year','NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales','Total_Sales']]

In [ ]:
country_group = year_sales.groupby(['Year']).sum().reset_index()

In [ ]:
country_melt = country_group.melt(id_vars='Year', value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales'], var_name='Country', value_name='Sales')

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(
    data = country_melt,
    x = "Country",
    y = "Sales",
    hue = "Year"
)
for p in ax.patches:
   ax.annotate(p.get_height().round(1), (p.get_x(), p.get_height()+1)) #p.get_height() : 높이 = count값, (p.get_x(), p.get_height()) : display 위치 

# 2. 국가별 선호 Genre

In [ ]:
genre_sales = df3[['Year', 'Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales','Total_Sales']]

In [ ]:
genre_sales

In [ ]:
genre_group = genre_sales.groupby(['Genre','Year']).sum().reset_index()

In [ ]:
genre_group

In [ ]:
def genre_barplot(df, genre):
    each_genre = df[df['Genre'] == genre]
    drop_index = each_genre.drop(['Genre'],axis=1)
    df_melt = drop_index.melt(id_vars='Year', value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales'], var_name='Country', value_name='Sales')

    plt.figure(figsize=(15, 10))
    ax = sns.barplot(
        data = df_melt,
        x = "Country",
        y = "Sales",
        hue = "Year"
    )
    # for p in ax.patches:
    #     ax.annotate(p.get_height().round(1), (p.get_x(), p.get_height()))
    ax.set_title(genre)

In [ ]:
genre_group['Genre'].unique()

In [ ]:
for i in range(12):
    genre_barplot(genre_group, genre_group['Genre'].unique()[i])

# 3. 국가별 선호 Platform

In [ ]:
platform_sales = df3[['Year', 'Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales','Total_Sales']]

In [ ]:
platform_group = platform_sales.groupby(['Platform','Year']).sum().reset_index()

In [ ]:
def platform_barplot(df, platform):
    each_platform = df[df['Platform'] == platform]
    drop_index = each_platform.drop(['Platform'],axis=1)
    df_melt = drop_index.melt(id_vars='Year', value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales'], var_name='Country', value_name='Sales')

    plt.figure(figsize=(15, 10))
    ax = sns.barplot(
        data = df_melt,
        x = "Country",
        y = "Sales",
        hue = "Year"
    )
    # for p in ax.patches:
    #     ax.annotate(p.get_height().round(1), (p.get_x(), p.get_height()))
    ax.set_title(platform)

In [ ]:
platform_group['Platform'].unique()

In [ ]:
platform_group

In [ ]:
for i in range(12):
    platform_barplot(platform_group, platform_group['Platform'].unique()[i])

# 4.2011~2015 국가별 장르 판매량 추이(기울기)

In [ ]:
def increase_genre(df, genre):
    sales_list = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales']

    each_genre = df[df['Genre'] == genre]
    drop_index = each_genre.drop(['Genre'],axis=1)
    df_melt = drop_index.melt(id_vars='Year', value_vars=sales_list, var_name='Country', value_name='Sales')

    temp = []
    for i in sales_list:
        each_country = df_melt[df_melt['Country'] == i]
        
        index = [x for x in range(len(each_country['Sales']))]
        data = each_country['Sales']
        
        coeffs = np.polyfit(index, list(data), 1)
        slope = coeffs[-2].round(2)

        temp.append(slope)
    return temp

In [ ]:
def increase_country(df):
    country_list = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales']
    temp = []
    for i in country_list:
        each_country = df[df['Country'] == i]
        index = [x for x in range(len(each_country['Sales']))]
        data = each_country['Sales']
        coeffs = np.polyfit(index, list(data), 1)
        slope = coeffs[-2].round(2)
        temp.append(slope)
    return temp

In [ ]:
total_increase = increase_country(country_melt)

In [ ]:
print('Genre',['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales'])
print('Total',total_increase)
for i in range(12):
    genre_name = genre_group['Genre'].unique()[i]
    print(genre_name, increase_genre(genre_group, genre_name))

In [ ]:
data = {'Genre': [], 'NA_Sales': [], 'EU_Sales': [], 'JP_Sales': [], 'Other_Sales': [], 'Total_Sales': []}
df_increase = pd.DataFrame(data=data)

In [ ]:
total_increase.insert(0, 'Total')
df_increase.loc[0] = total_increase

In [ ]:
for i in range(12):
    genre_name = genre_group['Genre'].unique()[i]
    row_data = increase_genre(genre_group, genre_name)
    row_data.insert(0,genre_name)
    df_increase.loc[i+1] = row_data

In [ ]:
df_increase

In [ ]:
melt = df_increase.melt(id_vars='Genre', value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales'], var_name='Country', value_name='Degree')
melt

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(
    data = melt,
    x = "Country",
    y = "Degree",
    hue = "Genre"
)

In [ ]:
def sort_increase(df, country):
    increase_mean = df[country].loc[1:12].mean()
    increase = df[df[country] >= increase_mean]
    return  increase.sort_values(by = country,ascending=False)

In [ ]:
sort_increase(df_increase, 'NA_Sales')

In [ ]:
sort_increase(df_increase, 'EU_Sales')

In [ ]:
sort_increase(df_increase, 'JP_Sales')

In [ ]:
sort_increase(df_increase, 'Other_Sales')

In [ ]:
sort_increase(df_increase, 'Total_Sales')